In [627]:
import numpy as np
import pandas as pd
import os

from sklearn.neighbors import KNeighborsClassifier

TITANIC_PATH = os.path.join("datasets", "titanic")


def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

In [628]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [629]:
print(train_data.shape, test_data.shape)

((891, 12), (418, 11))


In [630]:
from sklearn import preprocessing
def preprocess(data):
    le = preprocessing.LabelEncoder()
    cols = ['Name', 'Ticket', 'Cabin']
    data = data.drop(cols, axis=1)
    data['Age'] = data['Age'].interpolate()
    data["Sex"]=le.fit_transform(data["Sex"])
    data["Embarked"]=le.fit_transform(data["Embarked"])
    data['Fare'] = data['Fare'].interpolate()
    return data

train_data = preprocess(train_data)
test_data = preprocess(test_data)

In [631]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Embarked       891 non-null int64
dtypes: float64(2), int64(7)
memory usage: 62.7 KB


In [632]:
X_test  = test_data.drop('PassengerId',axis=1)
X_train = train_data.drop(['Survived','PassengerId'], axis=1)
y_train = train_data['Survived'].values

In [633]:
from sklearn.linear_model import LogisticRegression
LogR = LogisticRegression()
LogR.fit(X_train, y_train)
print(LogR.score(X_train, y_train))

0.7946127946127947


In [634]:
KNN = KNeighborsClassifier()
KNN.fit(X_train, y_train)
print(KNN.score(X_train, y_train))

0.797979797979798


In [635]:
y_pred = KNN.predict(X_test)

In [636]:
submission = pd.DataFrame({
        "PassengerId": test_data["PassengerId"],
        "Survived": y_pred
    })
submission.to_csv('./output/submission.csv', index=False)